#### Criando Schemas e Volume Tables para Armazenamento de Dados em DeltaTable

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS costumers_registrations.silver
COMMENT "Schema com Data Quality Aplicado";

CREATE VOLUME IF NOT EXISTS costumers_registrations.silver.registrations
COMMENT "Volume com Data Quality Aplicado para Cadastros";


#### Criando Delta Table

-----

Aplicando Data Quality

In [0]:
# Comparação de schemas esperado x real

from pyspark.sql.types import *
from pyspark.testing.utils import assertSchemaEqual

schema_expected = StructType([
    StructField('id', StringType(), True),
    StructField('nome', StringType(), True),
    StructField('data_nascimento', DateType(), True),
    StructField('cpf', StringType(),True),
    StructField('cep', StringType(), True),
    StructField('cidade', StringType(), True),
    StructField('estado', StringType(), True),
    StructField('pais', StringType(), True),
    StructField('genero', StringType(), True),
    StructField('telefone', StringType(), True),
    StructField('email', StringType(), True),
    StructField('data_cadastro', DateType(), True)
])

In [0]:
df_registrations = spark.read.parquet('/Volumes/costumers_registrations/raw/registrations')

actual_schema = df_registrations.schema


assertSchemaEqual(actual_schema, schema_expected)

print(f"Schema Esperado:\n {schema_expected}\n")
print(f"Schema Atual:\n {actual_schema}\n")
print("\nTeste de Schema Concluído\n")